In [25]:
# ======================================================
# PATH SETUP (NOTEBOOK)
# ======================================================
import sys
import os

ROOT_DIR = os.path.abspath("..")
if ROOT_DIR not in sys.path:
    sys.path.insert(0, ROOT_DIR)

# ======================================================
# IMPORTS
# ======================================================
from src.recommendation.hybrid_recommender import HybridRecommender
from src.recommendation.user_context_loader import UserContextLoader
from src.utils.io import load_json, load_pickle

from src.config.settings import (
    PRODUCT_DEPARTMENT_PATH,
    POPULAR_ITEMS_GLOBAL_PATH,
    POPULAR_ITEMS_BY_LIFECYCLE_PATH,
    POPULAR_ITEMS_BY_BEHAVIOR_PATH,
)


In [ ]:
# ======================================================
# LOAD STATIC DATA (OFFLINE MODE)
# ======================================================

product_department_map = {
    int(k): v
    for k, v in load_json(PRODUCT_DEPARTMENT_PATH).items()
}

popular_items_global = load_pickle(POPULAR_ITEMS_GLOBAL_PATH)
popular_items_by_lifecycle = load_pickle(POPULAR_ITEMS_BY_LIFECYCLE_PATH)
popular_items_by_behavior = load_pickle(POPULAR_ITEMS_BY_BEHAVIOR_PATH)

user_context_loader = UserContextLoader()

# ======================================================
# INIT RECOMMENDER
# ======================================================
recommender = HybridRecommender(
    product_department_map=product_department_map,
    user_context_loader=user_context_loader,
    popular_items_global=popular_items_global,
    popular_items_by_lifecycle=popular_items_by_lifecycle,
    popular_items_by_behavior=popular_items_by_behavior,
    popular_items_by_time={},  # optional (offline demo)
)

print("HybridRecommender ready (offline notebook mode)")


2026-01-26 13:30:41,770 | INFO | [UserContextLoader] Loaded 206,209 behavior assignments from behavior_cluster_assignments.csv
2026-01-26 13:30:41,820 | INFO | [UserContextLoader] Loaded 206,209 preference assignments from preference_cluster_assignments.csv
2026-01-26 13:30:41,887 | INFO | [UserContextLoader] Loaded 206,209 lifecycle assignments from lifecycle_assignments.csv
2026-01-26 13:30:42,394 | INFO | Rule index loaded | algo=fpgrowth v2 | schema_v=2 | contexts=511 | antecedents=254,630 | rules=668,550 | path=G:\VsCode\Python\ML\Suggest_Product\checkpoints\association_rules\fpgrowth_context_rule_index.pkl
2026-01-26 13:30:42,397 | INFO | Loaded FP-Growth rules | contexts=511
2026-01-26 13:30:42,397 | INFO | CandidateGenerator initialized | levels=['L1', 'L2', 'L3', 'L4', 'L5'] | max_ant_len=3
2026-01-26 13:30:42,398 | INFO | BehaviorAdjuster loaded | clusters=5
2026-01-26 13:30:42,400 | INFO | PreferenceFilter loaded | clusters=5
2026-01-26 13:30:42,400 | INFO | Ranker initializ

In [39]:
# ======================================================
# NOTEBOOK INPUT (MANUAL TEST)
# ======================================================

TEST_USER_ID = 123

TEST_BASKET = [
    12341, 35951, 14114, 25659, 35951
]

TEST_TIME_BUCKET = "morning"   # night | morning | afternoon | evening
TEST_IS_WEEKEND = False
TOP_K = 20


In [40]:
print("======================================")
print(" RECOMMENDATION REQUEST (NOTEBOOK)")
print("======================================")
print(f"user_id     : {TEST_USER_ID}")
print(f"basket      : {TEST_BASKET}")
print(f"time_bucket : {TEST_TIME_BUCKET}")
print(f"is_weekend  : {TEST_IS_WEEKEND}")
print(f"top_k       : {TOP_K}")


 RECOMMENDATION REQUEST (NOTEBOOK)
user_id     : 123
basket      : [12341, 35951, 14114, 25659, 35951]
time_bucket : morning
is_weekend  : False
top_k       : 20


In [41]:
recs = recommender.recommend(
    user_id=TEST_USER_ID,
    basket=TEST_BASKET,
    time_bucket=TEST_TIME_BUCKET,
    is_weekend=TEST_IS_WEEKEND,
    top_k=TOP_K,
)


2026-01-26 13:40:52,236 | INFO | [L1] contexts_available=336, contexts_matched=0, level_hits=0
2026-01-26 13:40:52,284 | INFO | [L2] contexts_available=134, contexts_matched=0, level_hits=0
2026-01-26 13:40:52,296 | INFO | [L3] contexts_available=32, contexts_matched=2, level_hits=7
2026-01-26 13:40:52,299 | INFO | [L4] contexts_available=8, contexts_matched=0, level_hits=0
2026-01-26 13:40:52,300 | INFO | [L5] contexts_available=1, contexts_matched=1, level_hits=0
2026-01-26 13:40:52,302 | WARNING | user_id=123 | rule_candidates=5 | fallback_needed=15
2026-01-26 13:40:52,314 | INFO | Top-20 ranked items: [(13176, 1.0), (21903, 0.9137), (47209, 0.4634), (21137, 0.36), (1, 0.3), (2, 0.3), (3, 0.3), (4, 0.3), (5, 0.3), (6, 0.3), (7, 0.3), (8, 0.3), (9, 0.3), (10, 0.3), (11, 0.3), (12, 0.3), (13, 0.3), (14, 0.3), (15, 0.3), (16, 0.3)]
2026-01-26 13:40:52,316 | INFO | user_id=123 | basket=5 | returned=20


In [42]:
import pandas as pd

print("\n--------------------------------------")
print(" RECOMMENDATION RESULT")
print("--------------------------------------")

if not recs:
    print("No recommendation returned")
else:
    df_rec = pd.DataFrame({
        "rank": range(1, len(recs) + 1),
        "product_id": recs
    })
    
    display(df_rec)

    for _, row in df_rec.iterrows():
        print(f"{row['rank']:02d}. Product {row['product_id']}")



--------------------------------------
 RECOMMENDATION RESULT
--------------------------------------


,rank,product_id
0,1,13176
1,2,21903
2,3,47209
3,4,21137
4,5,1
5,6,2
6,7,3
7,8,4
8,9,5
9,10,6


01. Product 13176
02. Product 21903
03. Product 47209
04. Product 21137
05. Product 1
06. Product 2
07. Product 3
08. Product 4
09. Product 5
10. Product 6
11. Product 7
12. Product 8
13. Product 9
14. Product 10
15. Product 11
16. Product 12
17. Product 13
18. Product 14
19. Product 15
20. Product 16
